In [1]:
from model import Network
from data import load_cifar10_dataloaders, WhiteningTransformation
import torch
import torchvision.transforms as transforms
import torch.nn as nn
from criterion import ReconstructImageFromFCLoss
import numpy as np
from metrics import lpips_gpu
import optuna

In [2]:
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(id))
print(torch.cuda.device(id))
print(torch.cuda.device_count())

0
NVIDIA RTX A5000
4


In [3]:
def objective(trial):
    # load to model
    model_config = './model_config/fc2_cocktail_party_cifar10_instance.json'
    checkpoint_path = './checkpoints/122123_fc2_cocktail_party_cifar10_pretraining_wout_bias_wout_normalization.pth'
    device = 'cuda:3' if torch.cuda.is_available() else 'cpu'
    model = Network(model_config)
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    model = model.to(device)

    # get val loader
    batch_size = 16
    data_path = './data'
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    # reproducibility
    random_seed = 2024

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    np.random.seed(random_seed)

    _, val_dataloader = load_cifar10_dataloaders(data_path, batch_size, transform)
    selected_val_batch_data, selected_val_batch_label = next(iter(val_dataloader))
    selected_val_batch_data = selected_val_batch_data.to(device)
    selected_val_batch_label = selected_val_batch_label.to(device)
    
    # receiving gradients
    model.zero_grad()
    criterion = nn.CrossEntropyLoss()
    output = model(selected_val_batch_data.reshape(batch_size, -1))
    loss = criterion(output, selected_val_batch_label)
    loss.backward()
    gradient_of_layers = []
    for param in model.parameters():
        gradient_of_layers.append(param.grad.data.clone().to('cpu'))
    print([x.size() for x in gradient_of_layers])
    
    whitening_transform = WhiteningTransformation()
    whitened_gradient = torch.from_numpy(whitening_transform.transform(gradient_of_layers[0].detach().numpy().T)).to(torch.float32).T
    whitened_gradient = whitened_gradient.to(device)
    
    # criterion output testing
    unmixing_matrix = torch.rand((selected_val_batch_data.size(0), gradient_of_layers[0].size(0)), requires_grad=True, device=device)
    t_param = trial.suggest_float('t_param', 0.00001, 10)
    total_variance_loss_param = trial.suggest_float('total_variance_loss_param', 0.00001, 10)
    mutual_independence_loss_param = trial.suggest_float('mutual_independence_loss_param', 0.00001, 10)
    reconstruction_loss = ReconstructImageFromFCLoss(32, 32, 3, t_param, total_variance_loss_param, mutual_independence_loss_param)
    optimizer = torch.optim.Adam([unmixing_matrix])
    
    for iter_idx in range(25000):
        optimizer.zero_grad()
        # out_score, non_gaussianity_score, total_variance_score, mutual_independence_score
        loss, _, _, _ = reconstruction_loss(unmixing_matrix, whitened_gradient)
        loss.backward()
        optimizer.step()

        if torch.isinf(loss).item() or torch.isnan(loss).item():
            return 100000000000
        
        if (iter_idx + 1) % 1000 == 0 or iter_idx == 0:
            print('loss: {}'.format(loss.item()))
            
    unmixing_matrix = unmixing_matrix.detach().to('cpu')
    whitened_gradient = whitened_gradient.detach().to('cpu')
    
    with torch.no_grad():
        lpips_match, lpips_is_positive, mean_lpips = lpips_gpu(unmixing_matrix @ whitened_gradient, selected_val_batch_data.detach().to('cpu'), return_matches=True)
        return mean_lpips

In [4]:
num_of_trials = 200
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=num_of_trials)

[I 2024-01-05 00:58:18,446] A new study created in memory with name: no-name-6f474409-d85f-4ded-99d5-18ccd04c64d7


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([256, 3072]), torch.Size([10, 256])]
loss: 664.6242065429688
loss: 20.15351676940918
loss: 8.529213905334473
loss: 7.841863632202148
loss: 7.795577526092529
loss: 7.742605686187744
loss: 7.682816982269287
loss: 7.611820697784424
loss: 7.52795934677124
loss: 7.442686557769775
loss: 7.3911662101745605
loss: 7.3797125816345215
loss: 7.377612590789795
loss: 7.376173496246338
loss: 7.3755717277526855
loss: 7.3753485679626465
loss: 7.3753981590271
loss: 7.37534761428833
loss: 7.375499248504639
loss: 7.37539005279541
loss: 7.3754563331604
loss: 7.375263214111328
loss: 7.375306129455566
loss: 7.3755950927734375
loss: 7.375235080718994
loss: 7.3753204345703125


[I 2024-01-05 00:59:23,876] Trial 0 finished with value: 0.2874278277158737 and parameters: {'t_param': 6.023753159076685, 'total_variance_loss_param': 3.3590249354986357, 'mutual_independence_loss_param': 7.369510631743043}. Best is trial 0 with value: 0.2874278277158737.


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([256, 3072]), torch.Size([10, 256])]
loss: 2379.619384765625
loss: 33.677268981933594
loss: 10.970717430114746
loss: 5.702253341674805
loss: 3.7203900814056396
loss: 2.9369101524353027
loss: 2.7819371223449707
loss: 2.6513969898223877
loss: 2.4864766597747803
loss: 2.282331705093384
loss: 2.066577911376953
loss: 1.9723314046859741
loss: 1.9651302099227905
loss: 1.9625765085220337
loss: 1.9624871015548706
loss: 1.9628570079803467
loss: 1.9626338481903076
loss: 1.9626833200454712
loss: 1.9629995822906494
loss: 1.9626829624176025
loss: 1.962792992591858
loss: 1.9628393650054932
loss: 1.9626249074935913
loss: 1.9628299474716187
loss: 1.9627842903137207
loss: 1.962646484375


[I 2024-01-05 01:00:29,019] Trial 1 finished with value: 0.2767477734014392 and parameters: {'t_param': 9.499789922799513, 'total_variance_loss_param': 6.95416819308266, 'mutual_independence_loss_param': 1.9593587384983362}. Best is trial 1 with value: 0.2767477734014392.


...

[I 2024-01-05 04:29:24,570] Trial 197 finished with value: 0.27191609237343073 and parameters: {'t_param': 7.912555347320816, 'total_variance_loss_param': 9.997627726359784, 'mutual_independence_loss_param': 9.152668879258208}. Best is trial 158 with value: 0.14164068852551281.


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([256, 3072]), torch.Size([10, 256])]
loss: 4789.31103515625
loss: 80.74982452392578
loss: 25.7380428314209
loss: 13.62399959564209
loss: 10.141251564025879
loss: 10.007418632507324
loss: 9.973427772521973
loss: 9.949573516845703
loss: 9.9315767288208
loss: 9.925400733947754
loss: 9.919448852539062
loss: 9.911341667175293
loss: 9.904147148132324
loss: 9.894545555114746
loss: 9.885452270507812
loss: 9.876116752624512
loss: 9.868416786193848
loss: 9.860747337341309
loss: 9.851470947265625
loss: 9.843684196472168
loss: 9.834575653076172
loss: 9.827102661132812
loss: 9.817046165466309
loss: 9.810369491577148
loss: 9.80219554901123
loss: 9.795100212097168


[I 2024-01-05 04:30:30,062] Trial 198 finished with value: 0.14676218386739492 and parameters: {'t_param': 8.291139138282276, 'total_variance_loss_param': 1.179264625098561, 'mutual_independence_loss_param': 9.759089810400164}. Best is trial 158 with value: 0.14164068852551281.


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([256, 3072]), torch.Size([10, 256])]
loss: 6739.1279296875
loss: 103.48534393310547
loss: 32.122169494628906
loss: 16.290380477905273
loss: 10.792948722839355
loss: 9.90726375579834
loss: 9.869407653808594
loss: 9.840315818786621
loss: 9.81986141204834
loss: 9.805747985839844
loss: 9.799589157104492
loss: 9.79403305053711
loss: 9.786819458007812
loss: 9.778185844421387
loss: 9.76981258392334
loss: 9.760405540466309
loss: 9.755183219909668
loss: 9.745205879211426
loss: 9.738173484802246
loss: 9.72893238067627
loss: 9.720261573791504
loss: 9.713041305541992
loss: 9.706167221069336
loss: 9.697771072387695
loss: 9.690627098083496
loss: 9.681384086608887


[I 2024-01-05 04:31:35,578] Trial 199 finished with value: 0.14452804485335946 and parameters: {'t_param': 8.764494850577996, 'total_variance_loss_param': 1.1693711351551876, 'mutual_independence_loss_param': 9.635483953881561}. Best is trial 158 with value: 0.14164068852551281.
